In [17]:
#%run "/Users/x644435/Documents/Good2Know/PythonLearn/Useful/UsefulFunc.py"
%run /nfs/science/users/xiaojiez/Python/UsefulPyFunc.py

Useful functions:
1. convert2Categ(df, threshold=5): to convert all category variables to dummy columns
2. flat2String(arg): to flattening arbitrary levels.
3. binary_perform(actual, pred): to print out binary classification performance.


In [15]:
### Save to a python script
# !jupyter nbconvert  Step0-Data.ipynb --to script


#Import popular libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


%matplotlib inline

### Set up display options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 300) # maximum display width 
pd.set_option('display.max_colwidth',120)

import os

print '### set up working path'
currpath = "/nfs/science/users/xiaojiez"
os.chdir(currpath)
print 'Current Working Directory:', os.getcwd(), '\n', 'List of Files:', os.listdir(currpath)
# os.remove(path)  # remove a file

### set up working path
Current Working Directory: /nfs/science/users/xiaojiez 
List of Files: ['Step0-Data-Template.ipynb', 'LCM', '.ipynb_checkpoints', 'Source', 'set_proxy_script.txt', 'FeatureRep', 'Learn', 'Rcode', 'Python', 'Junk', 'Shannon', 'TS', 'Churn', 'from_0199']


#### Read/write feather data

In [ ]:
import feather
path = 'my_data.feather'
feather.write_dataframe(df, path)
df = feather.read_dataframe(path)

#### Load R data into Python

In [ ]:
import rpy2.robjects as robjects
import pandas as pd

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

robjects.r['load']("Q20152DM.RData")

df = pandas2ri.ri2py(r['Q20152DM'])
df =  pd.DataFrame(df)

#### Read csv

In [ ]:
import pandas as pd
df = pd.read_csv('../Data/VALUE_P_TRAIN.csv')
print df.columns

#### Read SAS data

In [ ]:
import sys
sys.path.append("/Users/x644435/Documents/Good2Know/PythonLearn/sas7bdat-2.0.7/")
from sas7bdat import SAS7BDAT

# foo1 = SAS7BDAT('/Users/x644435/Documents/MOO/LCM-Digital/Data/ip_redc_vars.sas7bdat')
# df1 = foo1.to_data_frame()
# df1

# foo2 = SAS7BDAT('/Users/x644435/Documents/MOO/LCM-Digital/Data/temp.sas7bdat')
# df2 = foo2.to_data_frame()

# concatenate them horizontally
# frames = [df2, df1]
# df = pd.concat(frames,axis=1)
# df = df.drop(df.columns[[2,3,4,5,6,7,8,9,10]],1)
# df.shape

#### Read/write hdf5 data

In [ ]:
# store = pd.HDFStore('ip_redc_vars_9_13_2016.h5', mode='w')
# store['df1'] = df1  # save it
# store.close()
# df1.dtypes

store_r = pd.HDFStore('/Users/x644435/Documents/MOO/LCM-Digital/Python/ip_redc_vars_9_13_2016.h5', mode='r')

df = store_r['df1']  # load it
print 'Original df.shape:', df.shape

#### Get columns with levels 5 or less

In [ ]:
Col5less = pd.DataFrame([[col, df[col].dtype, len(df[col].unique())] for col in df.columns if len(df[col].unique()) <= 5], columns=['Colnames','DataType', 'nLevels'])
Col5less = Col5less.sort(['nLevels'])

print 'Columns with less than 5 distinct levels:\n', Col5less

#### Create a list with 1 level and drop off from dataframe

In [ ]:
RemoveList = Col5less[Col5less['nLevels']<=1][[0]].values.tolist()  # add column names with 1 level
RemoveList += ['HOUSEHOLD_ID','train']
RemoveList = list(str(RemoveList).translate(None,"[]u' ").split(','))
print '\n Columns with only 1 level + others, to be removed:\n', RemoveList

#drop columns
df.drop(RemoveList, axis=1, inplace=True)

#### Remove features with low variance

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
X = pd.DataFrame([[0, 0, 1], [0, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], [0, 1, 1]])
X.columns=['A','B','C']
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
print X, '\n', sel.fit_transform(X)

#### Create dummy variables

In [ ]:
#Create dummylist
Clist = list(df.select_dtypes(include=['O']).columns) #get all categorical columns
dummyList=Clist+['segment']
print "Dummifiy columns:", dummyList

df = pd.get_dummies(df, columns=dummyList, drop_first=True)
df.shape

#### Check if there are any non-float columns

In [ ]:
print 'Check non-float columns:', len(list (df.select_dtypes(exclude=['float']).columns))

#### Check if there are any more missing data

In [ ]:
NoMissing = df.isnull().sum()
NoMissing = NoMissing[NoMissing>0]

print 'Columns with non-zero missing data:\n\n', NoMissing

#### Extract a list of variables with certain prefix and apply log10+1 transformation

In [ ]:
recode_tmp  = [col for col in Cfeatures.columns if 'CPRE_NBSKTS' in col] +\
              [col for col in Cfeatures.columns if 'CPRE_SALE' in col]

Cfeatures[recode_tmp] = Cfeatures[recode_tmp].applymap(lambda x: np.log10(x+1))

#### Create training and testing data

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Xc_train, Xc_test, Xh_train, Xh_test, \
y_train, y_test,  = train_test_split(X, Xc, Xh, y, train_size=0.8, random_state=123)

#### Variable rescaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline


MinMax = MinMaxScaler(feature_range=(0, 1))
STD = StandardScaler(with_std=False)
pipe = Pipeline(steps=[('MinMax', MinMax), ('STD', STD)])


X_train = pipe.fit_transform(X_train)
X_test = pipe.transform(X_test)

#### Rescale a list of columns

In [ ]:
recode  = [col for col in Cfeatures.columns if 'CPRE_NBSKTS' in col] +\
              [col for col in Cfeatures.columns if 'CPRE_SALE' in col]
scaler = MinMaxScaler(feature_range=(0, 1))

X_train[to_recode] = scaler.fit_transform(X_train[to_recode])
X_test[to_recode] = scaler.transform(X_test[to_recode])